In [2]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from settings import *

### Specify input

In [7]:
## 15-min load data with BESS
path_input = os.path.join("results", f"load_with_BESS_15min.csv")
with_BESS = True

## 15-min load data without BESS
# path_input = os.path.join("results", f"load_without_BESS_15min.csv")
# with_BESS = False

# Change the start and end dates based on the data file
myd_start = "2022-05-31"
myd_end = "2022-06-01"

### Read 15-min loads

In [8]:
data = pd.read_csv(path_input)
   
dts = np.array(data['datetime'])
solar_loads = np.array(data['solar_load'])
building_loads = np.array(data['building_load'])
if with_BESS:
    BESS_outputs = np.array(data['BESS_output'])
else:
    BESS_outputs = np.zeros(len(data))
net_loads = building_loads - solar_loads - BESS_outputs

billing_interval = timedelta(minutes = 15)
billing_interval_hour = billing_interval.seconds/3600

### Run through each time step and record itemized costs

In [9]:
N = len(dts)
coste = {"Super Off-Peak": 0, "Off-Peak": 0, "On-Peak": 0}
costd = {"On-Peak": 0, "Non-Coincident": 0, "Generation": 0}
costE = {"Summer": coste.copy(), "Winter": coste.copy()}
costD = {"Summer": costd.copy(), "Winter": costd.copy()}
p_peaks = {"Summer": costd.copy(), "Winter": costd.copy()}
time_count = {"Summer": 0, "Winter": 0}

for i in range(N):
    dt = datetime.strptime(dts[i], "%Y%m%d-%H%M%S")
    
    # Determine weekday
    if dt.weekday() in [0, 1, 2, 3, 4]:
        # Determine month
        if dt.month in [3, 4]:
            peakHourTable = peakHourTable_weekdayMarApr
        else:
            peakHourTable = peakHourTable_weekday
    else:
        peakHourTable = peakHourTable_weekend
    
    # Determine season
    if dt.month in summerMonths:
        season = "Summer"
        energyChargeRef = energyChargeRef_Summer
        demandChargeRef = demandChargeRef_Summer
    else:
        season = "Winter"
        energyChargeRef = energyChargeRef_Winter
        demandChargeRef = demandChargeRef_Winter
    time_count[season] += 1
    
    # Get energy charge and demand charge at t
    peak_indicator = peakHourTable[dt.hour]
    peak_phase = peakNames[peak_indicator]
    ce = energyChargeRef[peak_indicator]
    costE[season][peak_phase] += ce * net_loads[i] * billing_interval_hour
    
    # Update peak loads
    if net_loads[i] > p_peaks[season]["Non-Coincident"]:
        p_peaks[season]["Non-Coincident"] = net_loads[i]
    if peak_phase == "On-Peak":
        if net_loads[i] > p_peaks[season]["On-Peak"]:
            p_peaks[season]["On-Peak"] = net_loads[i]

# Calculate various demand charges at the end of the billing period
for season in p_peaks.keys():
    p_peaks[season]["Non-Coincident"] = max(p_peaks["Summer"]["Non-Coincident"], p_peaks["Winter"]["Non-Coincident"])
    p_peaks[season]["Generation"] = p_peaks[season]["Non-Coincident"]
    
cds = {"Summer": {"On-Peak": cd_OnPeak_Summer,\
                  "Non-Coincident": cd_NonCoincident_Summer,\
                  "Generation": cd_Generation_Summer},
       "Winter": {"On-Peak": cd_OnPeak_Winter,\
                  "Non-Coincident": cd_NonCoincident_Winter,\
                  "Generation": cd_Generation_Winter}}
for season in costD.keys():
    ratio = time_count[season]/(time_count["Summer"] + time_count["Winter"])
    for typ in costD[season].keys():
        costD[season][typ] = cds[season][typ] * p_peaks[season][typ] * ratio

### Calculate total energy charge and demand charge

In [10]:
energyCharge = 0
for season in costE.keys():
    for peak_phase in costE[season].keys():
        energyCharge += costE[season][peak_phase]
        
demandCharge = 0
for season in costD.keys():
    for typ in costD[season].keys():
        demandCharge += costD[season][typ]

print(f"{myd_start}, {myd_end}")
print(f"Energy Charge: {energyCharge}")
print(f"Demand Charge: {demandCharge}")
print(f"Total Charge: {energyCharge + demandCharge}")

2022-05-31, 2022-06-01
Energy Charge: 41.75871621869415
Demand Charge: 2189.355879263023
Total Charge: 2231.114595481717
